## Combat Analyser

The CombatAnalyser is a tool to take a player and a target object, and to combine them to produce damage projections for a series of actions.

Whereas the sim uses the combat system's rules to generate simulation data of encounters, Zeal's core library instead projects average damages of each possible action.

This allows for rough appraisals of how ability damages change based on the player and target's stats (although there are still probably a few kinks to work out with regards to global damage factors in the calculations.

It should also allow for very robust evaluations of *relative* damage between actions, e.g. a SoB swing and a twist attempt.

I eventually want to feed these into a rotation generator algorithm that can use these numbers to arrive at things like optimal CS delay, by evaluating opportunity costs of the various actions.

In [1]:
import zeal.core as zc

## The library is all object oriented, with simple properties and methods.
## First we generate a player and target object.
p = zc.Player(weapon='torch', expertise=21)
t = zc.Target()

## Let's see their properties.
p.print_properties()
print()
t.print_properties()

Player properties:
--                   AP = 3400
--          crit chance = 0.35
--               weapon = torch
--              arm pen = 0
--            expertise = 21
--  chance to be dodged = 0.0125

Target properties:
--        base armor = 6200
--    modified armor = 1715
--             JotCr = True
--   expose weakness = 1200 agi


Let's now feed the player and target objects into the CombatAnalyser, a class designed to combine info on the two for calcs.

e.g. it figures out the total AP used in attacks from the player's AP, and the target's expose weakness.

It can then very quickly and efficiently rank the actions using quite basic calculations.

In [2]:
c = zc.CombatAnalyser(p, t)
c.rank_actions()

{'naked_swing': 0.6997633750474526,
 'sob_swing': 1.0,
 'soc_swing': 0.9568028283550427,
 'twist': 1.4259911679139037,
 'cs': 0.5678068038788655}

### High armor situation

Let's change our target to represent a Void Reaver fight, where we no longer benefit from Improved Expose Armor (but still benefit from 5-stack sunder armor).

How do the relative outputs change?

In [3]:
p2 = zc.Player(weapon='torch', expertise=21)  # same player properties
t2 = zc.Target(base_armor=8800, imp_ea=False)
t2.print_properties()
c2 = zc.CombatAnalyser(p2, t2)
c2.rank_actions()

Target properties:
--        base armor = 8800
--    modified armor = 4790
--             JotCr = True
--   expose weakness = 1200 agi


{'naked_swing': 0.6199286364912309,
 'sob_swing': 1.0,
 'soc_swing': 0.945316438561031,
 'twist': 1.5392648017453558,
 'cs': 0.5030267520005587}

### Expertise example

Let's also change the player's expertise and rerun the analysis.

As we can see, twists fall in relative value. This is because many of their damage components must survive 2-4 dodge rolls, which are more likely to fail with lower Expertise.

Having this kind of data quickly and readily available can be handy in theorycrafting or developing gearsets.

In [4]:
p.expertise = 24
c.rank_actions() 
p.expertise = 5
c.rank_actions()

{'naked_swing': 0.6978409298355416,
 'sob_swing': 1.0,
 'soc_swing': 0.956396969584237,
 'twist': 1.4298760038215372,
 'cs': 0.5664256618130245}

{'naked_swing': 0.7101824927676451,
 'sob_swing': 1.0,
 'soc_swing': 0.9589684257828687,
 'twist': 1.405292743743049,
 'cs': 0.5750829619836664}

# What next?

With this kind of data, it might be possible to dynamically generate rotations based on some kind of sensible algorithm, that can analytically arrive at things like the optimum CS delay, filler usage etc.

Given the relative worths of actions change depending on the fight conditions, it would be interesting to see if there are any situations where rotations change priority dynamically.

Indeed, we've already found one! Let's look at that projected VR fight again.

In [5]:
c2.rank_actions()
d = c2.rank_actions()
a = d['sob_swing'] + d['cs']
b = d['twist']
print(f'Damage from a SoB swing + CS = {a}')
print(f'Damage from a twist          = {b}')

# 1/2 is a twist then CS/SoB, 1/1 is 2 twists in the same time
ratio = 2*b / (a + b)

print(f'Relative benefit of 1/1 over 1/2 = {ratio}')

{'naked_swing': 0.6199286364912309,
 'sob_swing': 1.0,
 'soc_swing': 0.945316438561031,
 'twist': 1.5392648017453558,
 'cs': 0.5030267520005587}

Damage from a SoB swing + CS = 1.5030267520005587
Damage from a twist          = 1.5392648017453558
Relative benefit of 1/1 over 1/2 = 1.011911432255787


So in this (very simplified scenario, neglecting fillers etc.), 1/1 is about 1.2% better than the standard 1/2 rotation under these fight conditions.

Not the most earth-shattering revalation, a 1.2% difference on an older tier (that is probably obviated by filler usage etc.)...

But we can perhaps get the picture that having this information could be useful in theorycrafting.